In [32]:
import pandas as pd

df = pd.read_csv('all_france.csv')
df.head()

,Team_Flag_Src,Score_A,Score_B,Period,Time,Player,Action_Description,Total_Seconds,Number,Team_side,...,X_5_j_B,X_10_j_B,X_12_j_B,X_26_j_B,X_27_j_B,X_85_j_B,X_0_j_B,X_13_j_B,X_17_j_B,X_42_j_B
0,https://assets.fiba.basketball/image/upload/d_...,0,0,Q1,10:00,Victor Wembanyama,jump ball won,2400,32.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://assets.fiba.basketball/image/upload/d_...,0,0,Q1,10:00,Bruno Caboclo,jump ball lost,2400,51.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://assets.fiba.basketball/image/upload/d_...,0,0,Q1,09:39,Victor Wembanyama,3pt jump shot missed,2379,32.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://assets.fiba.basketball/image/upload/d_...,0,0,Q1,09:39,Marcelinho Huertas,defensive rebound,2379,9.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://assets.fiba.basketball/image/upload/d_...,0,2,Q1,09:33,Marcelinho Huertas,2pt floating jump shot made,2373,9.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df.game_code.unique()

array([117647, 117648, 117650])

In [35]:
from rotations_utils import get_dict_players_numbers
team_acronim = 'FRA'
dict_players_numbers  =  get_dict_players_numbers(df, team_acronim)
dict_players_numbers

{32.0: 'Victor Wembanyama',
 27.0: 'Rudy Gobert',
 85.0: 'Matthew Strazel',
 10.0: 'Evan Fournier',
 7.0: 'Guerschon Yabusele',
 1.0: 'Frank Ntilikina',
 5.0: 'Nicolas Batum',
 12.0: 'Nando De Colo',
 99.0: 'Bilal Coulibaly',
 26.0: 'Mathias Lessort',
 8.0: 'Isaia Cordinier',
 6.0: 'Andrew Albicy'}

In [36]:
# number = 8
# side = 'B'
# game_code = 117644

# on = df[((df[f'X_{number}_j_{side}'] == 1))&(df.game_code==game_code)][[ 'game_code','Score_A',	'Score_B',	'Period',	'Time',	'Player',	'Action_Description',	'Total_Seconds',	'Number',	'Team_side', 'Team', f'X_{number}_j_{side}']]
# on= df[(df.index.isin(on.index.tolist()))&(df.game_code==game_code)][[ 'game_code','Score_A',	'Score_B',	'Period',	'Time',	'Player',	'Action_Description',	'Total_Seconds',	'Number',	'Team_side', 'Team', f'X_{number}_j_{side}']]
# off= df[(~df.index.isin(on.index.tolist()))&(df.game_code==game_code)][[ 'game_code','Score_A',	'Score_B',	'Period',	'Time',	'Player',	'Action_Description',	'Total_Seconds',	'Number',	'Team_side', 'Team', f'X_{number}_j_{side}']]
# print(len(on), len(off), len(df[df.game_code==game_code]), len(on)+len(off)==len(df[df.game_code==game_code]))  

In [38]:
import pandas as pd
def calculate_value(row, team_acronim):
    action = row['Action_Description']
    team = row['Team']

    if 'made' not in action:
        return 0

    if '2pt' in action or 'dunk' in action or 'layup' in action or 'alley oop' in action or 'tip in' in action:
        value = 2
    elif 'free throw' in action:
        value = 1
    elif '3pt' in action:
        value = 3
    else:
        return 0

    if team == team_acronim:
        return value
    else:
        return -value



def calculate_contributions(df, dict_players_numbers, game_code, team_acronim):
    team_side_game_df = df[~df.Team.isna()][['game_code', 'Team', 'Team_side']].drop_duplicates().reset_index(drop=True)
    results = []

    for number, name in dict_players_numbers.items():

        number = int(number)
        side = team_side_game_df[(team_side_game_df.Team == team_acronim)&(team_side_game_df.game_code == game_code)]['Team_side'].values[0]

        on = df[((df[f'X_{number}_j_{side}'] == 1)) & (df.game_code == game_code)][['game_code', 'Score_A', 'Score_B', 'Period', 'Time', 'Player', 'Action_Description', 'Total_Seconds', 'Number', 'Team_side', 'Team', f'X_{number}_j_{side}']]
        if len(on) == 0:
            print('No data for player', name)
            continue

        on = df[(df.index.isin(on.index.tolist())) & (df.game_code == game_code)][['game_code', 'Score_A', 'Score_B', 'Period', 'Time', 'Player', 'Action_Description', 'Total_Seconds', 'Number', 'Team_side', 'Team', f'X_{number}_j_{side}']]
        off = df[(~df.index.isin(on.index.tolist())) & (df.game_code == game_code)][['game_code', 'Score_A', 'Score_B', 'Period', 'Time', 'Player', 'Action_Description', 'Total_Seconds', 'Number', 'Team_side', 'Team', f'X_{number}_j_{side}']]
        
        assert len(on) + len(off) == len(df[df.game_code == game_code])

        on['Plus_Minus'] = on.apply(calculate_value, team_acronim=team_acronim, axis=1)
        off['Plus_Minus'] = off.apply(calculate_value, team_acronim=team_acronim, axis=1)

        total_plus_minus_on = on['Plus_Minus'].sum()
        total_plus_minus_off = off['Plus_Minus'].sum()

        results.append({
            'Game_Code': game_code,
            'Player': name,
            'Number': number,
            'Total_Plus_Minus_On': total_plus_minus_on,
            'Total_Plus_Minus_Off': total_plus_minus_off
        })

    results_df = pd.DataFrame(results)
    return results_df


total_contributions = pd.DataFrame()
for game_cod in df.game_code.unique():
    team_acronim = 'FRA'
    contributions_df = calculate_contributions(df, dict_players_numbers, game_cod, team_acronim)
    total_contributions = pd.concat([total_contributions, contributions_df])
total_contributions[['Player', 'Total_Plus_Minus_On', 'Total_Plus_Minus_Off']].groupby('Player').sum().sort_values('Total_Plus_Minus_On', ascending=False).to_csv('on_off_france_1s.csv')
# total_contributions[['Player', 'Total_Plus_Minus_On', 'Total_Plus_Minus_Off']].groupby('Player').sum().sort_values('Total_Plus_Minus_On', ascending=False)

In [40]:
import pandas as pd
from itertools import combinations

def calculate_pair_contributions(df, dict_players_numbers, game_code, team_acronim):
    results = []

    player_numbers = list(dict_players_numbers.keys())

    for pair in combinations(player_numbers, 2):
        number1, number2 = pair
        number1 = int(number1)
        number2 = int(number2)
        name1 = dict_players_numbers[number1]
        name2 = dict_players_numbers[number2]

        side = 'B'

        # Calculate on-court contributions
        on = df[((df[f'X_{number1}_j_{side}'] == 1) & (df[f'X_{number2}_j_{side}'] == 1)) & (df.game_code == game_code)][['game_code', 'Score_A', 'Score_B', 'Period', 'Time', 'Player', 'Action_Description', 'Total_Seconds', 'Number', 'Team_side', 'Team', f'X_{number1}_j_{side}', f'X_{number2}_j_{side}']]
        # Calculate off-court contributions
        off = df[(~df.index.isin(on.index.tolist())) & (df.game_code == game_code)][['game_code', 'Score_A', 'Score_B', 'Period', 'Time', 'Player', 'Action_Description', 'Total_Seconds', 'Number', 'Team_side', 'Team', f'X_{number1}_j_{side}', f'X_{number2}_j_{side}']]

        assert len(on) + len(off) == len(df[df.game_code == game_code])

        if len(on) == 0:    
            continue

        on['Plus_Minus'] = on.apply(calculate_value, team_acronim=team_acronim, axis=1)
        off['Plus_Minus'] = off.apply(calculate_value, team_acronim=team_acronim, axis=1)

        total_plus_minus_on = on['Plus_Minus'].sum()
        total_plus_minus_off = off['Plus_Minus'].sum()

        results.append({
            'Player1': name1,
            'Player2': name2,
            'Number1': number1,
            'Number2': number2,
            'Total_Plus_Minus_On': total_plus_minus_on,
            'Total_Plus_Minus_Off': total_plus_minus_off
        })

    results_df = pd.DataFrame(results)
    return results_df

total_contributions = pd.DataFrame()
for game_cod in df.game_code.unique():
    team_acronim = 'FRA'
    pair_contributions_df = calculate_pair_contributions(df, dict_players_numbers, game_cod, team_acronim)
    total_contributions = pd.concat([total_contributions, pair_contributions_df])
total_contributions[['Player1', 'Player2', 'Total_Plus_Minus_On', 'Total_Plus_Minus_Off']].groupby(['Player1', 'Player2']).sum().sort_values('Total_Plus_Minus_On', ascending=False).to_csv('on_off_france_2s.csv')
# total_contributions[['Player1', 'Player2', 'Total_Plus_Minus_On', 'Total_Plus_Minus_Off']].groupby(['Player1', 'Player2']).sum().sort_values('Total_Plus_Minus_On', ascending=False)

In [42]:
import pandas as pd
from itertools import combinations

def calculate_triplet_contributions(df, dict_players_numbers, game_code, team_acronim):
    results = []

    player_numbers = list(dict_players_numbers.keys())

    for triplet in combinations(player_numbers, 3):
        number1, number2, number3 = triplet
        number1 = int(number1)
        number2 = int(number2)
        number3 = int(number3)
        name1 = dict_players_numbers[number1]
        name2 = dict_players_numbers[number2]
        name3 = dict_players_numbers[number3]

        side = 'B'

        # Calculate on-court contributions
        on = df[((df[f'X_{number1}_j_{side}'] == 1) & (df[f'X_{number2}_j_{side}'] == 1) & (df[f'X_{number3}_j_{side}'] == 1)) & (df.game_code == game_code)][['game_code', 'Score_A', 'Score_B', 'Period', 'Time', 'Player', 'Action_Description', 'Total_Seconds', 'Number', 'Team_side', 'Team', f'X_{number1}_j_{side}', f'X_{number2}_j_{side}', f'X_{number3}_j_{side}']]
        on = df[(df.index.isin(on.index.tolist())) & (df.game_code == game_code)][['game_code', 'Score_A', 'Score_B', 'Period', 'Time', 'Player', 'Action_Description', 'Total_Seconds', 'Number', 'Team_side', 'Team', f'X_{number1}_j_{side}', f'X_{number2}_j_{side}', f'X_{number3}_j_{side}']]

        # Calculate off-court contributions
        off = df[(~df.index.isin(on.index.tolist())) & (df.game_code == game_code)][['game_code', 'Score_A', 'Score_B', 'Period', 'Time', 'Player', 'Action_Description', 'Total_Seconds', 'Number', 'Team_side', 'Team', f'X_{number1}_j_{side}', f'X_{number2}_j_{side}', f'X_{number3}_j_{side}']]

        assert len(on) + len(off) == len(df[df.game_code == game_code])

        if len(on) == 0:    
            continue
        on['Plus_Minus'] = on.apply(calculate_value, team_acronim=team_acronim, axis=1)
        off['Plus_Minus'] = off.apply(calculate_value, team_acronim=team_acronim, axis=1)

        total_plus_minus_on = on['Plus_Minus'].sum()
        total_plus_minus_off = off['Plus_Minus'].sum()

        results.append({
            'Player1': name1,
            'Player2': name2,
            'Player3': name3,
            'Number1': number1,
            'Number2': number2,
            'Number3': number3,
            'Total_Plus_Minus_On': total_plus_minus_on,
            'Total_Plus_Minus_Off': total_plus_minus_off
        })

    results_df = pd.DataFrame(results)
    return results_df

# Example usage:

total_contributions = pd.DataFrame()
for game_cod in df.game_code.unique():
    team_acronim = 'FRA'
    triplet_contributions_df = calculate_triplet_contributions(df, dict_players_numbers, game_cod, team_acronim)
    total_contributions = pd.concat([total_contributions, triplet_contributions_df])

total_contributions[['Player1', 'Player2', 'Player3', 'Total_Plus_Minus_On', 'Total_Plus_Minus_Off']].groupby(['Player1', 'Player2', 'Player3']).sum().sort_values('Total_Plus_Minus_On', ascending=False).to_csv('on_off_france_3s.csv')
# total_contributions[['Player1', 'Player2', 'Player3', 'Total_Plus_Minus_On', 'Total_Plus_Minus_Off']].groupby(['Player1', 'Player2', 'Player3']).sum().sort_values('Total_Plus_Minus_On', ascending=False)